In [114]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import TargetEncoder

In [115]:
df = pd.read_json("./public/diary.json")

In [116]:
df.head()

,course,name,quantity,date
0,fruit,Frutta fresca,1,2023-09-19
1,firstCourse,Pasta al pomodoro,1,2023-09-19
2,firstCourse,Nasello gratinato,1,2023-09-19
3,firstCourse,Insalata verde,4,2023-09-19
4,fruit,Frutta fresca,1,2023-09-20


In [117]:
df = df.replace({ 1: 10, 2: 7, 3: 3, 4: 0 })

In [119]:
grouped_by_date = df.groupby("date")
grouped_by_date_mean = grouped_by_date.mean("quantity")
grouped_by_date_mean.index = grouped_by_date_mean.index + pd.DateOffset(1)

In [120]:
grouped_by_name = df.groupby("name")

In [121]:
df = df.merge(grouped_by_name.mean("quantity"), right_index=True, left_on="name")
df = df.rename({"quantity_x": "target", "quantity_y": "mean"}, axis=1)

In [122]:
df = df.merge(grouped_by_date_mean, right_index=True, left_on="date")
df = df.rename({"quantity": "lag"}, axis=1)

In [132]:
grouped_by_name.apply(lambda row: print(row))

    course    name  quantity       date
38   fruit  Banana        10 2023-09-29
58   fruit  Banana        10 2023-10-06
79   fruit  Banana        10 2023-10-13
100  fruit  Banana        10 2023-10-20
124  fruit  Banana        10 2023-10-27
125  fruit  Banana        10 2023-10-27
139  fruit  Banana         3 2023-11-06
157  fruit  Banana        10 2023-11-10
162  fruit  Banana         0 2023-11-13
183  fruit  Banana        10 2023-11-20
201  fruit  Banana        10 2023-11-24
205  fruit  Banana        10 2023-11-27
216  fruit  Banana        10 2023-12-04
229  fruit  Banana        10 2023-12-11
265  fruit  Banana        10 2023-12-28
274  fruit  Banana        10 2024-01-02
304  fruit  Banana        10 2024-01-12
327  fruit  Banana        10 2024-01-29
          course                             name  quantity       date
147  firstCourse  Bastoncini artigianali di pesce        10 2023-11-08
223  firstCourse  Bastoncini artigianali di pesce        10 2023-12-06
277  firstCourse  Bastoncin

/var/folders/xy/np4nx7d164b4tzvs90j55qlw0000gp/T/ipykernel_72305/2154812946.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_by_name.apply(lambda row: print(row))


""


In [123]:
df

,course,name,target,date,mean,lag
4,fruit,Frutta fresca,10,2023-09-20,10.000000,7.50
5,firstCourse,Pasta alla crema di rapa rossa,10,2023-09-20,10.000000,7.50
6,firstCourse,Petto di pollo al limone,10,2023-09-20,6.500000,7.50
7,firstCourse,Erbette all'olio,0,2023-09-20,0.000000,7.50
8,fruit,Yougurt bianco,3,2023-09-20,4.900000,7.50
...,...,...,...,...,...,...
323,firstCourse,Zucca al forno,3,2024-01-19,1.000000,6.75
328,fruit,Mela,7,2024-01-30,7.813333,5.75
329,firstCourse,Minestra di cous cous,7,2024-01-30,9.250000,5.75
330,firstCourse,Bocconcini di tacchino allo yogurt,10,2024-01-30,10.000000,5.75


In [124]:
train, test = train_test_split(df)

In [125]:
X_train = train.drop("target", axis=1)
y_train = train["target"]

In [126]:
X_test = test.drop("target", axis=1)
y_test = test["target"]

In [127]:
encoder = TargetEncoder(target_type="continuous").set_output(transform="pandas")

In [128]:
X_train["course"] = encoder.fit_transform(X_train[["name"]], y_train)

,course,name,date,mean,lag
97,10.000000,Minestra di riso integrale con i carciofi,2023-10-20,8.500000,7.500000
154,4.405615,Pera,2023-11-09,2.875000,6.600000
211,7.304054,Polenta con ragù di vitellone,2023-11-29,0.000000,6.000000
317,6.748752,Mela,2024-01-18,7.813333,6.750000
193,7.159561,Mela,2023-11-23,7.813333,9.250000
...,...,...,...,...,...
328,7.159561,Mela,2024-01-30,7.813333,5.750000
86,6.939597,Insalata verde e rossa,2023-10-17,0.000000,6.666667
91,6.932886,Erbette all'olio,2023-10-18,0.000000,4.000000
170,3.985296,Yougurt bianco,2023-11-15,4.900000,2.500000
